In [ ]:
# import the required libraries

import pandas as pd
import numpy as np

df_invoices = pd.read_csv(
    r'filepath')

df_exceptions = pd.read_csv(
    r'filepath')

print("Libraries and Excel files read into pandas")

In [ ]:
# the function defines the conditions under which the Invoice Type column will display "Purchase Order" or "Others"

def f(row):
    if row['PO Id'] != 'Unclassified':
        val = 'Purchase Order'
    else:
        val = 'Others'
        
    return val

# add new Invoice Type column
df_invoices['Invoice Type'] = df_invoices.apply(f,axis=1)

print(df_invoices.shape)
df_invoices.head()


In [ ]:
# replace 'Unclassified' in date columns

df_invoices['Invoice Date Created - Date'] = df_invoices['Invoice Date Created - Date'].replace('Unclassified', np.nan)
df_invoices['Invoice Date - Date'] = df_invoices['Invoice Date - Date'].replace('Unclassified', np.nan)
df_invoices['Invoice Submit Date - Date'] = df_invoices['Invoice Submit Date - Date'].replace('Unclassified', np.nan)
df_invoices['Approved Date - Date'] = df_invoices['Approved Date - Date'].replace('Unclassified', np.nan)
df_invoices['Reconciled Date - Date'] = df_invoices['Reconciled Date - Date'].replace('Unclassified', np.nan)
df_invoices['Paid Date - Date'] = df_invoices['Paid Date - Date'].replace('Unclassified', np.nan)
df_invoices['Accounting Date'] = df_invoices['Accounting Date'].replace('Unclassified', np.nan)
df_invoices['Received Date - Date'] = df_invoices['Received Date - Date'].replace('Unclassified', np.nan)

# convert all dates to datetime

df_invoices['Invoice Date Created - Date'] = pd.to_datetime(df_invoices['Invoice Date Created - Date'])
df_invoices['Invoice Date - Date'] = pd.to_datetime(df_invoices['Invoice Date - Date'])
df_invoices['Invoice Submit Date - Date'] = pd.to_datetime(df_invoices['Invoice Submit Date - Date'])
df_invoices['Approved Date - Date'] = pd.to_datetime(df_invoices['Approved Date - Date'])
df_invoices['Reconciled Date - Date'] = pd.to_datetime(df_invoices['Reconciled Date - Date'])
df_invoices['Paid Date - Date'] = pd.to_datetime(df_invoices['Paid Date - Date'])
df_invoices['Accounting Date'] = pd.to_datetime(df_invoices['Accounting Date'])
df_invoices['Received Date - Date'] = pd.to_datetime(df_invoices['Received Date - Date'])

# confirm the date types changed

print(df_invoices.shape)
print("Invoice Date Created: ", df_invoices['Invoice Date Created - Date'].dtypes)
print("Invoice Date: ", df_invoices['Invoice Date - Date'].dtypes)
print("Invoice Submit Date: ", df_invoices['Invoice Submit Date - Date'].dtypes)
print("Approved Date: ", df_invoices['Approved Date - Date'].dtypes)
print("Reconciled Date: ", df_invoices['Reconciled Date - Date'].dtypes)
print("Paid Date: ",df_invoices['Paid Date - Date'].dtypes)
print("Accounting Date: ", df_invoices['Accounting Date'].dtypes)
print("Received Date: ", df_invoices['Received Date - Date'].dtypes)

In [ ]:
# Invoice to Submit
# caclulate difference in 'days'
df_invoices["Invoice to Submit"] = df_invoices['Invoice Submit Date - Date'].sub(
    df_invoices['Invoice Date - Date'], axis = 0)

# convert to float so difference displays as numeric
df_invoices['Invoice to Submit'] = df_invoices['Invoice to Submit']/np.timedelta64(1,'D')

# repeat for remaining date metrics

# Submit to Approved IR
df_invoices['Submit to Approved IR Date'] = df_invoices['Accounting Date'].sub(
    df_invoices['Invoice Submit Date - Date'], axis = 0)

df_invoices['Submit to Approved IR Date'] = df_invoices['Submit to Approved IR Date']/np.timedelta64(1, 'D')

# IR Date to Approved Date
df_invoices['IR Date to Approved Date'] = df_invoices['Approved Date - Date'].sub(
    df_invoices['Accounting Date'], axis= 0)

df_invoices['IR Date to Approved Date'] = df_invoices['IR Date to Approved Date']/np.timedelta64(1, 'D')

# Invoice Received to Approved Date
df_invoices['Invoice Received to Approved Date'] = df_invoices['Invoice Submit Date - Date'].sub(
    df_invoices['Received Date - Date'], axis= 0)

df_invoices['Invoice Received to Approved Date'] = df_invoices['Invoice Received to Approved Date']/np.timedelta64(1, 'D')

print(df_invoices.shape)
df_invoices.head()

In [ ]:
# clean the exceptions df so only the invoice id and exceptions columns remain
# this will simplify the join

df_exceptions = df_exceptions[['Invoice ID', 'Exception Types - Exception',
       'Exception Types - Invoice Exception Level',
       'Exception Types - Invoice Exception Type']]

# perform the join

invoice_report = pd.merge(df_invoices,
                         df_exceptions,
                         on = 'Invoice ID',
                         how = 'left')

print(df_invoices.shape)
invoice_report.head()

In [ ]:
invoice_report.to_excel(
    r'filepath',
    index=False)

print('report read to excel')